In [1]:
%pip install -U transformers datasets peft evaluate tf-keras sacrebleu rouge_score pycocoevalcap nltk

  Using cached transformers-4.48.1-py3-none-any.whl.metadata (44 kB)
  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
  Using cached peft-0.14.0-py3-none-any.whl.metadata (13 kB)
  Using cached evaluate-0.4.3-py3-none-any.whl.metadata (9.2 kB)
  Using cached tf_keras-2.18.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached sacrebleu-2.5.1-py3-none-any.whl.metadata (51 kB)
  Using cached rouge_score-0.1.2-py3-none-any.whl
  Using cached pycocoevalcap-1.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py311-none-any.whl.metadata (7.2 kB)
  Using cached accelerate-1.3.0-py3-none-any.whl.metadata (19 kB)
  Using cached tensorflow-2.18.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_

In [2]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, DataCollatorForLanguageModeling, DataCollatorForSeq2Seq, AutoTokenizer, get_linear_schedule_with_warmup
from datasets import load_dataset
import evaluate
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
from tqdm.auto import tqdm
import re
import nltk
import pandas as pd
from itertools import groupby
from datasets import Dataset
# Update the training and evaluation loop
from torch.utils.data import DataLoader


2025-01-26 19:26:57.564414: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-26 19:26:57.596936: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737919617.618988     285 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737919617.625455     285 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-26 19:26:57.657761: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [14]:
ranks = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]
alpha=32
weight_decay = 0.01
dropout_prob = 0.1
label_smooth = 0.1
length_penalty = 0.9
max_length = 128
batch_size=8
lr=2e-4
num_epochs=5
num_warmup_steps=500
train_set_size=None
eval_set_size=None

In [5]:
# https://huggingface.co/docs/transformers/preprocessing
nltk.download("punkt", quiet=True)
model_name = "gpt2-large"
tokenizer = AutoTokenizer.from_pretrained(model_name) #GPT2Tokenizer.from_pretrained('gpt2-medium')
tokenizer.padding_side = 'left'
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
encoded_input = tokenizer(["This is an example sentence for the rank evaluation task."], truncation=True, padding='max_length', max_length=128)
print(encoded_input)
print(tokenizer.decode(encoded_input["input_ids"][0]))

{'input_ids': [[50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 1212, 318, 281, 1672, 6827, 329, 262, 4279, 12660, 4876, 13]], 'attention_mask': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [6]:
#from datasets import disable_caching
#disable_caching()
dataset = load_dataset("e2e_nlg")

train_set = dataset["train"]
val_set = dataset["validation"]
test_set = dataset["test"]
train_set[0]

{'meaning_representation': 'name[The Vaults], eatType[pub], priceRange[more than £30], customer rating[5 out of 5], near[Café Adriatic]',
 'human_reference': 'The Vaults pub near Café Adriatic has a 5 star rating.  Prices start at £30.'}

In [7]:
def tokenize_function(examples):
    return tokenizer(examples["meaning_representation"], truncation=True, padding='max_length', max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/42061 [00:00<?, ? examples/s]

Map:   0%|          | 0/4672 [00:00<?, ? examples/s]

Map:   0%|          | 0/4693 [00:00<?, ? examples/s]

In [8]:
# Grouping function for E2E NLG test dataset
def group_e2e_test_data(test_data):
    df = pd.DataFrame(test_data)
    df.sort_values(by='meaning_representation', inplace=True)
    grouped = df.groupby('meaning_representation')['human_reference'].apply(list).reset_index()
    grouped_dataset = Dataset.from_pandas(grouped)
    return grouped_dataset

def preprocess_e2e(examples):
    inputs = examples['meaning_representation']
    targets = examples['human_reference']
    texts = [inp + ' | ' + tgt + " " + tokenizer.eos_token for inp, tgt in zip(inputs, targets)]
    model_inputs = tokenizer(texts, truncation=True)
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    return model_inputs

def preprocess_e2e_eval(examples):
    inputs = examples['meaning_representation']
    targets = examples['human_reference']
    texts = [inp + ' | ' for inp in inputs]
    model_inputs = tokenizer(texts, truncation=True)
    model_inputs["meaning_representation"] = texts
    model_inputs["human_reference"] = targets
    return model_inputs

In [9]:
# needs to be installed after dataset has been loaded
%pip install nlg-metricverse

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached nlg_metricverse-0.9.9-py3-none-any.whl.metadata (23 kB)
  Using cached datasets-2.9.0-py3-none-any.whl.metadata (19 kB)
  Using cached fire-0.7.0-py3-none-any.whl
  Using cached nltk-3.7-py3-none-any.whl.metadata (2.8 kB)
  Using cached click-8.1.3-py3-none-any.whl.metadata (3.2 kB)
  Using cached syllables-1.0.9-py3-none-any.whl.metadata (2.4 kB)
  Using cached typing-3.7.4.3-py3-none-any.whl
  Using cached textstat-0.7.4-py3-none-any.whl.metadata (14 kB)
  Using cached codecarbon-2.1.4-py3-none-any.whl.metadata (18 kB)
  Using cached validators-0.34.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached bert_score-0.3.13-py3-none-any.whl.metadata (15 kB)
  Using cached pyemd-1.0.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.6 kB)
  Using cached pynvml-12.0.0-py3-none-any.whl.metadata (5.4 kB)
  Using cached fuzzywuzzy-0.18.0-py2.py3-none-any.whl.metadata (4.9 kB)
  Using cached dill-0.3.6-py3-none-any.whl.metadata (9.8 kB)
  Using cached respons

In [10]:
from nlgmetricverse import NLGMetricverse, load_metric

bleu = evaluate.load('bleu')
rouge = evaluate.load('rouge')
nist = evaluate.load('nist_mt')
cider = NLGMetricverse(metrics=load_metric("cider"))
meteor = NLGMetricverse(metrics=load_metric("meteor"))

In [11]:
def custom_collate_fn(batch):
    human_references = [item['human_reference'] for item in batch]
    meaning_representations = [item['meaning_representation'] for item in batch]
    # Remove 'human_reference' before using data_collator
    batch = [{'input_ids': item['input_ids'], 'attention_mask': item['attention_mask']} for item in batch]
    batch = data_collator(batch)
    batch['human_reference'] = human_references
    batch['meaning_representation'] = meaning_representations
    return batch

def generate_predictions(test_dataloader, model, tokenizer, length_penalty):
    model.eval()
    predictions = []
    references = []
    progress_bar = tqdm(test_dataloader, desc="Generating predictions")
    for batch in progress_bar:
        # Use only the meaning representation as input
        input_ids = batch['input_ids'].to(model.device)
        attention_mask = batch['attention_mask'].to(model.device)
        meaning_representations = batch['meaning_representation']
        human_references = batch['human_reference']
        with torch.no_grad():
            output_ids = model.generate(
                input_ids,
                attention_mask=attention_mask,
                num_beams=10,
                length_penalty=length_penalty,
                repetition_penalty=1.0,
                no_repeat_ngram_size=4,
                max_new_tokens=64,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )
        for i, output in enumerate(output_ids):
            prediction = tokenizer.decode(output, skip_special_tokens=True)
            input_text = meaning_representations[i]
            #if i < 5:
            #    print("Prediction:")
            #    print(prediction[0])
            #    print("References:")
            #    print(human_references[i])
            prediction = prediction.strip()
            predictions.append(format_outputs(prediction))
            references.append([format_outputs(human_reference) for human_reference in human_references[i]]) 
    return predictions, references

def format_outputs(text):
    text = text.lower()
    text = ' '.join(re.split('(\W)', text))
    text = text.split()
    text = ' '.join(text)
    return text

In [12]:
test_data = group_e2e_test_data(dataset['test'])
preprocess_function = preprocess_e2e
preprocess_function_eval = preprocess_e2e_eval

train_data = dataset['train']

train_tokenized = train_data.map(preprocess_function, batched=True, remove_columns=train_data.column_names)
if train_set_size is not None:
    train_tokenized = train_tokenized.select(range(train_set_size))
test_tokenized = test_data.map(preprocess_function_eval, batched=True, remove_columns=test_data.column_names)
if eval_set_size is not None:
    test_tokenized = test_tokenized.select(range(eval_set_size))
# Data collator and DataLoaders
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, return_tensors="pt", padding=True)

train_dataloader = DataLoader(train_tokenized, shuffle=True, batch_size=batch_size, collate_fn=data_collator)
test_dataloader = DataLoader(test_tokenized, batch_size=batch_size, collate_fn=custom_collate_fn)

Map:   0%|          | 0/42061 [00:00<?, ? examples/s]

Map:   0%|          | 0/630 [00:00<?, ? examples/s]

In [ ]:

for rank in ranks:
    peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=rank,
    lora_alpha=alpha,
    target_modules=["c_attn"],
    lora_dropout=dropout_prob,
    init_lora_weights="gaussian",
    bias="none"
    )
    
    model = GPT2LMHeadModel.from_pretrained(model_name)
    # FREEZE WEIGHTS
    #for param in model.parameters():
    #    param.requires_grad = False
    if rank != 0:
        model = get_peft_model(model, peft_config)
    model = model.to(device)
    with torch.no_grad():
        model.resize_token_embeddings(len(tokenizer))
    model.config.pad_token_id = tokenizer.pad_token_id
    print("Evaluation for Rank: " + str(rank))
    if rank != 0:
        print(model.print_trainable_parameters())

    # Optimizer and scheduler
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    num_training_steps = num_epochs * len(train_dataloader)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)
    
    # Training loop
    for epoch in range(num_epochs):
        model.train()
        progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")
        for batch in progress_bar:
            inputs = {key: val.to(device) for key, val in batch.items()}
            outputs = model(**inputs)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            progress_bar.set_postfix(loss=loss.item())
    
    # Evaluation
    predictions, references = generate_predictions(test_dataloader, model, tokenizer, length_penalty=length_penalty)
    bleu_score = bleu.compute(predictions=predictions, references=references)
    meteor_score = meteor(predictions=predictions, references=references)
    rouge_score = rouge.compute(predictions=predictions, references=references)
    nist_score = nist.compute(predictions=predictions, references=references)
    cider_score = cider(predictions=predictions, references=references)

    test_metrics = {
        'bleu': bleu_score['bleu'],
        'meteor': meteor_score['meteor']['score'],
        'rouge_l': rouge_score['rougeL'],
        'nist': nist_score['nist_mt'],
        'cider': cider_score['cider']['score']
    }

    print(test_metrics)

/opt/conda/lib/python3.11/site-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


Evaluation for Rank: 1
trainable params: 184,320 || all params: 774,215,680 || trainable%: 0.0238
None


Epoch 1/1:   0%|          | 0/5258 [00:00<?, ?it/s]

# PyTorch Trainer Framework
Initially, this notebook was aimed to be based on the transformer's Trainer API (https://huggingface.co/docs/transformers/main_classes/trainer). However, we faced OutOfMemoryExceptions during evaluation. Also, we were not able to properly adjust the hyperparameters as described by LoRA.

In [ ]:
%pip install -U transformers sentence-transformers datasets peft evaluate tf-keras sacrebleu rouge_score nltk #  fsspec==2023.9.2 pycocoevalcap git+https://github.com/salaniz/pycocoevalcap git+https://github.com/disi-unibo-nlp/nlg-metricverse#-quickstart

In [16]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling, DataCollatorForSeq2Seq, AutoTokenizer, AutoModelForCausalLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
from evaluate import load
import pandas as pd
from datasets import Dataset
import nltk
import evaluate
import numpy as np
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from transformers import get_linear_schedule_with_warmup


In [5]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [6]:
# https://huggingface.co/docs/transformers/preprocessing
nltk.download("punkt", quiet=True)
model_name = "gpt2-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name) #GPT2Tokenizer.from_pretrained('gpt2-medium')
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"
encoded_input = tokenizer(["This is an example sentence for the rank evaluation task."], truncation=True, padding='max_length', max_length=128)
print(encoded_input)
print(tokenizer.decode(encoded_input["input_ids"][0]))

{'input_ids': [[50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 1212, 318, 281, 1672, 6827, 329, 262, 4279, 12660, 4876, 13]], 'attention_mask': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [7]:
dataset = load_dataset("e2e_nlg")

train_set = dataset["train"]
val_set = dataset["validation"]
test_set = dataset["test"]
train_set[0]

{'meaning_representation': 'name[The Vaults], eatType[pub], priceRange[more than £30], customer rating[5 out of 5], near[Café Adriatic]',
 'human_reference': 'The Vaults pub near Café Adriatic has a 5 star rating.  Prices start at £30.'}

In [8]:
def tokenize_function(examples):
    return tokenizer(examples["meaning_representation"], truncation=True, padding='max_length', max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/42061 [00:00<?, ? examples/s]

Map:   0%|          | 0/4672 [00:00<?, ? examples/s]

Map:   0%|          | 0/4693 [00:00<?, ? examples/s]

In [9]:
# https://huggingface.co/docs/evaluate/transformers_integrations
# https://medium.com/@prashanth.ramanathan/fine-tuning-a-pre-trained-gpt-2-model-and-performing-inference-a-hands-on-guide-57c097a3b810
#def preprocess_function(examples):
#    model_inputs = tokenizer(examples["meaning_representation"], truncation=True, padding='max_length', max_length=128)
#    labels = tokenizer(text_target=examples["human_reference"], truncation=True, padding='max_length', max_length=128)
#
#    model_inputs["labels"] = labels["input_ids"]
#    return model_inputs

def preprocess_function(examples):
    model_inputs = tokenizer(examples["meaning_representation"], truncation=True, padding='max_length', max_length=128)

    examples["prediction"] = "Meaning Representation: " + examples["meaning_representation"] + " Human Reference: " + examples["human_reference"]
    return tokenizer(examples["prediction"], truncation=True, padding='max_length', max_length=128)

def preprocess_function_alt(examples):
    model_inputs = tokenizer(examples["meaning_representation"], truncation=True, padding='max_length', max_length=128)

    examples["prediction"] = "Meaning Representation: " + examples["meaning_representation"] + " Human Reference: " + examples["human_reference"]
    tokenized = tokenizer(examples["prediction"], truncation=True, padding='max_length', max_length=128)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

def preprocess_function_val(examples):
    model_inputs = tokenizer("Meaning Representation: " + examples["meaning_representation"] + " Human Reference: ", truncation=True, padding='max_length', max_length=128)
    labels = tokenizer(text_target=examples["human_reference"], truncation=True, padding='max_length', max_length=128)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

preprocessed_datasets = dict()
preprocessed_datasets["train"] = dataset["train"].map(preprocess_function)
preprocessed_datasets["validation"] = dataset["validation"].map(preprocess_function_val)
preprocessed_datasets["test"] = dataset["test"].map(preprocess_function_val)

print("Validation example:")
print("Reference:")
print(tokenizer.decode(preprocessed_datasets['validation'][0]["labels"]))
print("Model Input:")
print(tokenizer.decode(preprocessed_datasets['validation'][0]["input_ids"]))
preprocessed_datasets["train"] = preprocessed_datasets["train"].remove_columns(["prediction", "meaning_representation", "human_reference"])

Map:   0%|          | 0/42061 [00:00<?, ? examples/s]

Map:   0%|          | 0/4672 [00:00<?, ? examples/s]

Map:   0%|          | 0/4693 [00:00<?, ? examples/s]

Validation example:
Reference:
<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endof

In [44]:
rank=64
alpha=32
dropout=0.1

In [45]:
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=rank,
    lora_alpha=alpha,
    #target_modules=["c_attn"],
    lora_dropout=dropout,
    init_lora_weights="gaussian",
    bias="none"
)

model = GPT2LMHeadModel.from_pretrained(model_name, max_length=128)
# FREEZE WEIGHTS
#for param in model.parameters():
#    param.requires_grad = False
model = get_peft_model(model, peft_config)
model = model.to(device)
#with torch.no_grad():
#    model.resize_token_embeddings(len(tokenizer))
#model.config.pad_token_id = tokenizer.pad_token_id
model.print_trainable_parameters()

/opt/conda/lib/python3.11/site-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 6,291,456 || all params: 361,114,624 || trainable%: 1.7422


In [13]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
training_args = TrainingArguments(output_dir="logs_gpt2_rank_e2e_00",
                                         #eval_strategy="steps",
                                         #learning_rate=2e-4,
                                         per_device_train_batch_size=64,
                                         #gradient_accumulation_steps=4,
                                         #weight_decay=0.01,
                                         #save_total_limit=3,
                                         #num_train_epochs=2,
                                         remove_unused_columns=False,
                                         warmup_steps=500,
                                         #weight_decay=0.01,
                                         #fp16=True,
                                         #predict_with_generate=True,
                                         #eval_steps=50,
                                         logging_dir="./logs_gpt2_rank_e2e_00",
                                         logging_steps=100,
                                         max_steps=100
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=preprocessed_datasets["train"],
    #eval_dataset=preprocessed_datasets["validation"].shuffle().select(range(128)),
    tokenizer=tokenizer,
    data_collator=data_collator,
    #preprocess_logits_for_metrics=preprocess_logits_for_metrics
)

trainer.train()

In [27]:
# Generation example
inputs = tokenizer('Meaning Representation: name[Cocum], eatType[coffee shop], food[Chinese], priceRange[high], customer rating[1 out of 5], familyFriendly[yes] Human Reference: ', return_tensors='pt')
outputs = model.generate(**inputs.to(device), max_length=128, num_return_sequences=1)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(generated_text)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Meaning Representation: name[Cocum], eatType[coffee shop], food[Chinese], priceRange[high], customer rating[1 out of 5], familyFriendly[yes] Human Reference:  "I'm a coffee shop owner, and I'm a family friend."
The first thing I noticed was that the name was a little different. I was expecting a coffee shop, but instead I got a coffee shop. I'm not sure why, but I'm guessing that the name was chosen because it's a coffee shop. I'm not sure if it's because I'm a coffee shop owner,


In [39]:
print(references[1])
print()
print(predictions[1])

Blue Spice is a coffee shop in city centre.

 Blue Spice is a coffee shop in the city centre with a price range of £20-25-25. It is located in the centre of the city centre. It has a customer rating of 3 out of 5, and it is located near the corner of the City Centre. It is called Blue Spice. It is
